In [1]:
# A final Python notebook (storage.ipynb)
# The final, clean dataset stored in a way that is optimized for analytics and future data loads.

import pandas as pd
import numpy as np
import os
import json
from datetime import datetime
from sqlalchemy import create_engine


# Load raw data

RAW_FILE = "cwur_rankings_raw.csv"
OUTPUT_DIR = "data_cleaned"
VERSION = "v2025"

os.makedirs(OUTPUT_DIR, exist_ok=True)

try:
    df = pd.read_csv(RAW_FILE)
    print(f"Loaded raw dataset: {len(df)} rows")
except FileNotFoundError:
    raise SystemExit(f"Could not find {RAW_FILE}. Run acquisition first!")

# Hnadle missing data

missing_rank = df["Rank"].isna().sum()
print(f"Missing Rank values: {missing_rank}")

# Drop rows with missing rank (critical for ranking analysis)
df = df.dropna(subset=["Rank"])

# Fill missing scores with 0 (justified: not scored)
df["Score"] = df["Score"].fillna("0")

# Data type conversion

def clean_numeric(value):
    """Convert text values (like '101-150', 'N/A') to numeric."""
    if pd.isna(value):
        return np.nan
    value = str(value).replace(",", "").replace("%", "").replace("†", "").strip()
    if "-" in value:
        try:
            low, high = value.split("-")
            return (float(low) + float(high)) / 2  # Convert range to midpoint
        except:
            return np.nan
    if value in ["N/A", ""]:
        return np.nan
    try:
        return float(value)
    except ValueError:
        return np.nan

df["Rank"] = df["Rank"].apply(clean_numeric).astype("Int64", errors="ignore")
df["Score"] = df["Score"].apply(clean_numeric)

# Feature engineering

region_map = {
    "United States": "North America", "Canada": "North America", "Mexico": "North America",
    "United Kingdom": "Europe", "Germany": "Europe", "France": "Europe", "Italy": "Europe",
    "Spain": "Europe", "Netherlands": "Europe", "Sweden": "Europe", "Switzerland": "Europe",
    "China": "Asia", "Japan": "Asia", "India": "Asia", "South Korea": "Asia", "Singapore": "Asia",
    "Australia": "Oceania", "New Zealand": "Oceania",
    "South Africa": "Africa", "Egypt": "Africa", "Nigeria": "Africa", "Kenya": "Africa", "Uganda": "Africa",
    "Brazil": "South America", "Argentina": "South America", "Chile": "South America", "Colombia": "South America"
}

# Map countries to global regions, fill unmapped as 'Other'
df["Global_Region"] = df["Country"].map(region_map).fillna("Other")

# Normalize score between 0–1
if df["Score"].notna().any():
    df["Overall_Score_Normalized"] = (df["Score"] - df["Score"].min()) / (df["Score"].max() - df["Score"].min())
else:
    df["Overall_Score_Normalized"] = np.nan

# Optimized storage formats

# Versioned CSV for readability
csv_path = os.path.join(OUTPUT_DIR, f"cwur_rankings_{VERSION}.csv")
df.to_csv(csv_path, index=False)

# Parquet format for analytics (compressed, columnar)
parquet_path = os.path.join(OUTPUT_DIR, f"cwur_rankings_{VERSION}.parquet")
df.to_parquet(parquet_path, index=False)

print(f"\nClean data saved:")
print(f" - CSV: {csv_path}")
print(f" - Parquet: {parquet_path}")

# Save to Analytics Database

# SQLite DB for future merges, queries, dashboards

engine = create_engine("sqlite:///university_rankings.db")
df.to_sql("cwur_rankings", con=engine, if_exists="replace", index=False)
print("Data also loaded into SQLite database (university_rankings.db)")

# Metadata summary for future loads

metadata = {
    "dataset_version": VERSION,
    "record_count": len(df),
    "columns": df.columns.tolist(),
    "date_processed": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "regions": df["Global_Region"].value_counts().to_dict(),
    "score_range": [float(df["Score"].min()), float(df["Score"].max())],
    "normalized_score_range": [float(df["Overall_Score_Normalized"].min()), float(df["Overall_Score_Normalized"].max())],
}

metadata_path = os.path.join(OUTPUT_DIR, f"metadata_{VERSION}.json")
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=4)

print(f"\nMetadata summary saved to {metadata_path}")
print("\nData wrangling and optimization completed successfully!")

Loaded raw dataset: 10000 rows
Missing Rank values: 0


ImportError: Unable to find a usable engine; tried using: 'pyarrow', 'fastparquet'.
A suitable version of pyarrow or fastparquet is required for parquet support.
Trying to import the above resulted in these errors:
 - Missing optional dependency 'pyarrow'. pyarrow is required for parquet support. Use pip or conda to install pyarrow.
 - Missing optional dependency 'fastparquet'. fastparquet is required for parquet support. Use pip or conda to install fastparquet.

In [2]:
# A) RAW SUMMARY (before cleaning)
import pandas as pd

RAW = "cwur_rankings_raw.csv"  # adjust path if needed
raw = pd.read_csv(RAW)

def to_num(x):
    s = str(x)
    # strip commas/%/markers
    for tok in [",", "%", "†"]:
        s = s.replace(tok, "")
    # handle ranges like '101-150' -> midpoint
    if "-" in s:
        a, b = s.split("-", 1)
        try:
            return (float(a) + float(b)) / 2.0
        except:
            return pd.NA
    # remove trailing text like '1Top 0.1%'
    # keep leading digits if any
    import re
    m = re.match(r"^\s*(\d+(\.\d+)?)", s)
    if m:
        return float(m.group(1))
    try:
        return float(s)
    except:
        return pd.NA

raw_stats = {}
for col in ["Rank","Score"]:
    vals = raw[col].map(to_num)
    raw_stats[col] = vals.describe()[["count","mean","std","min","max"]].round(3)

raw_summary = pd.DataFrame(raw_stats)
print("RAW SUMMARY (Rank & Score):")
print(raw_summary)

# Optional: quick peek of missingness
print("\nMissingness (raw):")
print(raw[["Rank","Score"]].isna().sum())

RAW SUMMARY (Rank & Score):
            Rank      Score
count  10000.000  10000.000
mean    1000.500     71.676
std      577.379     89.211
min        1.000      1.000
max     2000.000    346.000

Missingness (raw):
Rank     0
Score    0
dtype: int64


In [3]:
# B) CLEAN SUMMARY (after wrangling)
import pandas as pd

CLEAN = "cwur_rankings_cleaned.csv"  # adjust path if needed
clean = pd.read_csv(CLEAN)

clean_stats = {}
for col in ["Rank","Score","Overall_Score_Normalized"]:
    if col in clean.columns:
        clean_stats[col] = clean[col].describe()[["count","mean","std","min","max"]].round(3)

clean_summary = pd.DataFrame(clean_stats)
print("CLEAN SUMMARY (Rank, Score, Normalized):")
print(clean_summary)

# Region distribution (nice to include)
if "Global_Region" in clean.columns:
    region_counts = clean["Global_Region"].value_counts(dropna=False)
    print("\nRows by Global_Region:")
    print(region_counts)

CLEAN SUMMARY (Rank, Score, Normalized):
       Rank      Score  Overall_Score_Normalized
count   0.0  10000.000                 10000.000
mean    NaN     71.676                     0.205
std     NaN     89.211                     0.259
min     NaN      1.000                     0.000
max     NaN    346.000                     1.000

Rows by Global_Region:
Asia             3735
Other            2695
Europe           2460
South America     420
North America     255
Oceania           235
Africa            200
Name: Global_Region, dtype: int64


In [4]:
# C) Save both summaries to a single Excel + Markdown for easy pasting
# Robust against:
# - Missing files or different paths
# - Missing Excel writer engines
# - Missing 'tabulate' for to_markdown

import re
from pathlib import Path
import pandas as pd

# ---- locate files safely ----
def find_file(candidates):
    for p in candidates:
        p = Path(p)
        if p.exists() and p.is_file():
            return p
    return None

RAW_CANDIDATES = [
    "cwur_rankings_raw.csv",
    "data_cleaned/cwur_rankings_v2025.csv",
    "1_raw_university_rankings.csv",
]
CLEAN_CANDIDATES = [
    "cwur_rankings_cleaned.csv",
    "data_cleaned/cwur_rankings_v2025.csv",  # if you used the storage output as “clean”
    "2_clean_university_rankings.csv",
]

raw_path = find_file(RAW_CANDIDATES)
clean_path = find_file(CLEAN_CANDIDATES)

if raw_path is None:
    raise SystemExit(
        "Could not find RAW file. Looked for: "
        + ", ".join(map(str, RAW_CANDIDATES))
        + "\nRun acquisition first or adjust the path list."
    )
if clean_path is None:
    raise SystemExit(
        "Could not find CLEAN file. Looked for: "
        + ", ".join(map(str, CLEAN_CANDIDATES))
        + "\nRun wrangling first or adjust the path list."
    )

print(f"RAW -> {raw_path}")
print(f"CLEAN -> {clean_path}")

raw = pd.read_csv(raw_path)
clean = pd.read_csv(clean_path)

# ---- robust numeric parser ----
def to_num(x):
    """Parses messy numeric strings:
    - strips commas, %, †
    - handles ranges '101-150' -> midpoint
    - pulls leading number from tokens like '1Top 0.1%'
    """
    if pd.isna(x):
        return pd.NA
    s = str(x).strip().replace(",", "").replace("%", "").replace("†", "")
    # range 'a-b' -> midpoint
    if "-" in s:
        a, b = s.split("-", 1)
        try:
            return (float(a) + float(b)) / 2.0
        except Exception:
            pass
    # leading number (covers '1Top 0.1%')
    m = re.match(r"^\s*([0-9]+(\.[0-9]+)?)", s)
    if m:
        try:
            return float(m.group(1))
        except Exception:
            return pd.NA
    # plain float fallback
    try:
        return float(s)
    except Exception:
        return pd.NA

def summarize_numeric(df, cols):
    out = {}
    for c in cols:
        if c not in df.columns:
            continue
        series = df[c]
        # if object-like, map with to_num; else coerce
        if series.dtype == "object":
            vals = series.map(to_num)
        else:
            vals = pd.to_numeric(series, errors="coerce")
        desc = vals.describe()
        # Ensure keys exist even if few rows
        for k in ["count", "mean", "std", "min", "max"]:
            if k not in desc.index:
                desc.loc[k] = pd.NA
        out[c] = desc[["count", "mean", "std", "min", "max"]].round(3)
    return pd.DataFrame(out)

raw_summary = summarize_numeric(raw, ["Rank", "Score"])
clean_cols = [c for c in ["Rank", "Score", "Overall_Score_Normalized"] if c in clean.columns]
clean_summary = summarize_numeric(clean, clean_cols)

print("\nRAW SUMMARY (Rank & Score):\n", raw_summary, sep="")
print("\nCLEAN SUMMARY (Rank, Score, Overall_Score_Normalized):\n", clean_summary, sep="")

# ---- exports ----
out_xlsx = Path("summary_stats.xlsx")
region_counts = None
if "Global_Region" in clean.columns:
    region_counts = clean["Global_Region"].value_counts(dropna=False).to_frame("count")

# Excel export (no engine specified -> pandas will choose available writer)
try:
    with pd.ExcelWriter(out_xlsx) as xw:
        raw_summary.to_excel(xw, sheet_name="Raw_Summary")
        clean_summary.to_excel(xw, sheet_name="Clean_Summary")
        if region_counts is not None:
            region_counts.to_excel(xw, sheet_name="Region_Counts")
    print(f"\n Saved Excel: {out_xlsx}")
except Exception as e:
    # Fallback: save CSVs instead if Excel writer missing
    raw_summary.to_csv("raw_summary.csv")
    clean_summary.to_csv("clean_summary.csv")
    if region_counts is not None:
        region_counts.to_csv("region_counts.csv")
    print("\n Excel writer not available. Saved CSV fallbacks:")
    print(" - raw_summary.csv")
    print(" - clean_summary.csv")
    if region_counts is not None:
        print(" - region_counts.csv")
    print(f"   Reason: {type(e)._name_}: {e}")

# Markdown export (fallback if 'tabulate' missing)
def to_markdown_safe(df):
    try:
        return df.to_markdown()  # needs 'tabulate'
    except Exception:
        # Simple pipe table fallback
        header = "| metric | " + " | ".join(df.columns) + " |\n"
        sep = "|" + " --- |" * (len(df.columns) + 1) + "\n"
        body = ""
        for idx, row in df.iterrows():
            body += f"| {idx} | " + " | ".join(str(v) for v in row.values) + " |\n"
        return header + sep + body

Path("raw_summary.md").write_text(to_markdown_safe(raw_summary), encoding="utf-8")
Path("clean_summary.md").write_text(to_markdown_safe(clean_summary), encoding="utf-8")
print("Saved Markdown: raw_summary.md, clean_summary.md")

RAW -> cwur_rankings_raw.csv
CLEAN -> cwur_rankings_cleaned.csv

RAW SUMMARY (Rank & Score):
            Rank      Score
count  10000.000  10000.000
mean    1000.500     71.676
std      577.379     89.211
min        1.000      1.000
max     2000.000    346.000

CLEAN SUMMARY (Rank, Score, Overall_Score_Normalized):
       Rank      Score  Overall_Score_Normalized
count   0.0  10000.000                 10000.000
mean    NaN     71.676                     0.205
std     NaN     89.211                     0.259
min     NaN      1.000                     0.000
max     NaN    346.000                     1.000

 Saved Excel: summary_stats.xlsx
Saved Markdown: raw_summary.md, clean_summary.md


In [5]:
# D) Hypothesis — print and save safely
from pathlib import Path

hypothesis = ("Hypothesis: After standardization and de-duplication, the distribution of scores remains stable "
    "while rank variance decreases due to the removal of malformed and duplicate records.")
print(hypothesis)

Path("hypothesis.txt").write_text(hypothesis, encoding="utf-8")
print(" Saved: hypothesis.txt")

Hypothesis: After standardization and de-duplication, the distribution of scores remains stable while rank variance decreases due to the removal of malformed and duplicate records.
 Saved: hypothesis.txt
